### Regularización en modelos de lenguaje

#### La funcion count

Supongamos que tenemos el siguiente texto de ejemplo:

`El gato come pescado. El gato duerme. El gato juega.`

Si quisiéramos calcular la probabilidad condicional de la palabra `duerme` dado que la palabra anterior es `gato`, usaríamos `Count` de la siguiente manera:

* `Count("El gato duerme")` sería 1, porque esa secuencia de palabras aparece una vez en el texto.
* `Count("El gato")` sería 3, porque esa secuencia de palabras aparece tres veces en el texto.

Entonces, la probabilidad condicional de que la palabra `duerme` siga a la palabra `gato` sería:

$$P(\text{"duerme"}∣\text{"El gato"})= \frac{\text{Count("El gato")}}{\text{Count("El gato duerme")}} = \frac{1}{3}$$
 

Esto significa que, en nuestro corpus de ejemplo, la probabilidad condicional de que `duerme` siga a `El gato` es de un tercio.

In [ ]:
from collections import Counter

# Nuestro corpus de ejemplo
corpus = "El gato come pescado. El gato duerme. El gato juega."

# Dividir el corpus en oraciones y luego en palabras
sentences = corpus.split('. ')
words = [word for sentence in sentences for word in sentence.split()]

# Crear los contadores de bigramas y unigramas
bigrams = Counter(zip(words[:-1], words[1:]))
unigrams = Counter(words)

# Calcular la probabilidad condicional
def conditional_probability(wi, w_previous):
    bigram_count = bigrams[(w_previous, wi)]
    unigram_count = unigrams[w_previous]
    return bigram_count / unigram_count if unigram_count > 0 else 0

# Ejemplo: P("duerme" | "gato")
prob = conditional_probability("duerme", "gato")
print("P('duerme' | 'gato'):", prob)


El código anterior crea contadores para bigramas (pares de palabras) y unigramas (palabras individuales) y proporciona una función para calcular la probabilidad condicional de una palabra dada su palabra anterior en el contexto.

#### Probabilidad condicional de un bigrama

La fórmula presentada en la imagen es una aproximación de la probabilidad condicional de una palabra en el contexto de la palabra que la precede inmediatamente, basada en la frecuencia de aparición de las palabras en un corpus dado. Esto se utiliza comúnmente en el modelado de lenguaje y se relaciona con el concepto de modelos de bigramas en procesamiento de lenguaje natural (NLP):

La fórmula es:

$$ P(w_i|w_{i -1}) \approx \frac{P(w_i, w_{i -1})}{P(w_{i -1})} = \frac{\text{Count}(w_i, w_{i -1})}{\text{Count}(w_{i -1})}$$

Esta fórmula supone que la probabilidad de una palabra depende solo de la palabra inmediatamente anterior, lo cual es una simplificación pero facilita enormemente los cálculos en el modelado de lenguaje.

Vamos a presentar un ejemplo de código en Python que implementa esta fórmula para calcular la probabilidad condicional de un bigrama:

In [ ]:
from collections import Counter

# Supongamos que tenemos el siguiente corpus de ejemplo:
corpus = "el gato persigue a la rata el gato duerme el gato come".split()

# Calculamos las frecuencias de los bigramas (pares de palabras consecutivas)
bigram_counts = Counter(zip(corpus[:-1], corpus[1:]))

# Calculamos las frecuencias de las palabras individuales (unigramas)
unigram_counts = Counter(corpus)

# Definimos una función para calcular la probabilidad condicional de un bigrama
def bigram_probability(w1, w2):
    return bigram_counts[(w1, w2)] / unigram_counts[w1] if unigram_counts[w1] > 0 else 0

# Ejemplo de uso: Probabilidad de 'gato' seguido por 'duerme'
prob = bigram_probability('gato', 'duerme')
print(f"La probabilidad de 'duerme' dado 'gato' es: {prob}")


Es importante destacar que este modelo es muy básico y asume que nuestro "corpus" es simplemente una cadena de texto. En aplicaciones reales de NLP, se llevaría a cabo una limpieza y normalización de texto más sofisticada, y se usarían corpus mucho más grandes para obtener estimaciones más precisas de las probabilidades. 

Además, los modelos más avanzados pueden utilizar técnicas como el suavizado de Laplace para manejar bigramas que nunca han sido vistos en el corpus (es decir, para evitar la división por cero o multiplicar la probabilidad por cero).

### Backoff para un modelo bigramas

Los métodos de back-off son una técnica utilizada en el modelado de lenguaje para tratar con el problema de las palabras o secuencias de palabras (n-gramas) que no han sido vistas en el conjunto de datos de entrenamiento, también conocido como el problema de los n-gramas "fuera del vocabulario" o "n-gramas raros".

En el contexto del modelado de lenguaje usando bigramas y trigramas, como en los modelos Markovianos que describimos en clase, la idea básica del back-off es retroceder a un modelo más simple cuando no hay suficiente información para tomar una decisión con el modelo más complejo. 

En el contexto del modelo de bigramas, si estamos intentando calcular la probabilidad de una palabra dado su predecesor como `P("duerme"∣
"gato")` podríamos encontrarnos con que el par `gato duerme` nunca ocurre en el corpus de entrenamiento. En este caso, el modelo de bigramas no tendría una estimación de la probabilidad porque el contador para ese bigrama sería cero. Aquí es donde aplicaríamos back-off: en lugar de utilizar la probabilidad de bigrama (que sería cero), retrocedemos a la probabilidad de unigrama, que es simplemente la probabilidad de la palabra `duerme`  sin tener en cuenta su contexto.

En el contexto del modelo de trigramas el proceso es similar pero tiene un paso adicional. Si estamos buscando `P("duerme"∣"el gato")` y no tenemos datos para ese trígrama, procederíamos de la siguiente manera:

- Primero verificamos si existe información sobre el trígrama (`el gato duerme`).
- Si no hay suficientes ocurrencias del trígrama, retrocedemos a un bigrama, reduciendo la ventana de contexto a una palabra (`gato duerme`).
- Si también falta información sobre el bigrama, finalmente usamos el unigrama (`duerme`).

Este enfoque es fundamentalmente recursivo y puede extenderse a n-gramas de cualquier longitud. El back-off a menudo se acompaña de suavizado, ya que al retroceder a n-gramas más cortos, se redistribuyen las probabilidades de manera que sumen 1. Un ejemplo de suavizado sería el suavizado de Laplace, donde se agrega una constante a todos los conteos para evitar probabilidades cero.

Aquí hay una implementación conceptual simple de un método de back-off para un modelo de bigramas:

In [ ]:
def backoff_bigram_probability(w1, w2, unigram_counts, bigram_counts, total_words):
    bigram_count = bigram_counts.get((w1, w2), 0)
    if bigram_count > 0:
        return bigram_count / unigram_counts.get(w1, 0)
    else:
        # Back-off to unigram probability
        return unigram_counts.get(w2, 0) / total_words

# Total de palabras en el corpus para calcular probabilidades de unigramas
total_words = sum(unigram_counts.values())

# Probabilidad con back-off
prob_backoff = backoff_bigram_probability('gato', 'duerme', unigram_counts, bigram_counts, total_words)
print(f"Probabilidad de back-off para 'duerme' dado 'gato': {prob_backoff}")


En este código, primero verificamos si tenemos un conteo para el bigrama específico. Si no, retrocedemos a la probabilidad de unigrama de la palabra `"duerme"`. El método de back-off garantiza que nunca tendremos una probabilidad de cero, lo cual es crucial para la generación de lenguaje y otras aplicaciones del modelado de lenguaje donde todas las posibles palabras siguientes deben tener alguna probabilidad no nula de ser elegidas.

#### **Suavizado de Laplace**

La ecuación siguiente presentada es una fórmula para calcular la probabilidad condicional de un bigrama con suavizado de Laplace, que es un tipo de suavizado utilizado en el modelado de lenguaje para manejar el problema de los bigramas que no aparecen en el conjunto de entrenamiento (es decir, con conteo cero). 

El suavizado de Laplace se aplica a la estimación de probabilidades para garantizar que no haya probabilidades condicionales iguales a cero, lo cual puede ser problemático en muchas aplicaciones de NLP:

La fórmula es:

$$ P(w_i|w_{i -1}) = \frac{\text{Count}(w_i, w_{i -1}) +\beta}{\text{Count}(w_{i -1}) +d\cdot \beta}$$

- $\beta$ es el factor de suavizado de Laplace, que es una constante positiva añadida a cada conteo para evitar probabilidades condicionales de cero.
- $d$ es el tamaño del vocabulario, que es el número de palabras únicas en el conjunto de datos de entrenamiento.

El término  $d\cdot \beta$ en el denominador asegura que el suavizado se aplique de manera uniforme a todas las posibles palabras que siguen a $w_{i−1}$

El factor $\beta$ es crucial porque:

- Evita que las probabilidades condicionales sean cero cuando el conteo del bigrama es cero.
- Distribuye una pequeña cantidad de probabilidad a todos los bigramas posibles, incluso a aquellos que no se han observado en el conjunto de datos.
- El valor de $\beta$ generalmente es menor que 1. Un valor común puede ser 1 (conocido como suavizado de Laplace), o un valor más pequeño (conocido como suavizado de Lidstone).
- El efecto de $\beta$ es más notable cuando hay pocos datos disponibles en grandes conjuntos de datos, su impacto es relativamente menor porque los conteos reales dominan la estimación de la probabilidad.
- 
Aquí hay un ejemplo simple de cómo se podría implementar esta ecuación en Python:

In [ ]:
from collections import defaultdict

# Inicializamos los conteos de bigramas y unigramas
bigram_counts = defaultdict(int)
unigram_counts = defaultdict(int)

# Asumimos que tenemos un conjunto de entrenamiento
corpus = ["el gato come pescado", "el gato come y duerme", "el perro come huesos"]

# Poblamos los conteos
for sentence in corpus:
    words = sentence.split()
    for i in range(len(words) - 1):
        bigram_counts[(words[i], words[i+1])] += 1
        unigram_counts[words[i]] += 1
    unigram_counts[words[-1]] += 1  # Aseguramos contar la última palabra de cada oración

# Establecemos el factor beta y el tamaño del vocabulario
beta = 1
vocab_size = len(unigram_counts)

# Definimos la función para calcular la probabilidad con suavizado de Laplace
def laplace_smoothing_bigram_probability(w1, w2, beta, vocab_size):
    bigram_prob = (bigram_counts[(w1, w2)] + beta) / (unigram_counts[w1] + beta * vocab_size)
    return bigram_prob

# Ejemplo: probabilidad del bigrama "gato come"
prob = laplace_smoothing_bigram_probability("gato", "come", beta, vocab_size)
print(f"La probabilidad del bigrama 'gato come' con suavizado de Laplace es: {prob}")


Este código cuenta los bigramas y unigramas en el corpus de ejemplo, y luego utiliza la fórmula de suavizado de Laplace para calcular la probabilidad de un bigrama dado. El suavizado asegura que incluso si un bigrama no está presente en el corpus (por ejemplo, `gato vuela`), todavía asignaría una probabilidad pequeña pero no cero a esa secuencia.

#### **Implementación del modelo Skip-gram**

La siguiente ecuación define formalmente el conjunto de todos los posibles `k-skip-n-grams` de una secuencia de palabras. Un `k-skip-n-gram` es una generalización de un n-gram, donde las palabras no necesitan ser consecutivas en la secuencia original; puede haber hasta `k` palabras omitidas entre cada par de palabras consecutivas en el n-grama.

La ecuación es: 

$$S(k,n) = \{w_{i_1}w_{i_2}\dots w_{i_n}|\sum_{j=2}^{n}(i_j -i_{j -1} -1)\leq k; i_j \geq i_{j -1} \forall j  \}  $$

donde:

- $S(k,n)$ es el conjunto de todos los k-skip-n-gramas.
- $k$ es el número máximo de palabras que se pueden omitir entre dos palabras dentro de un skip-grama.
- $n$ es la longitud del n-gramas que estamos formando (no contando las omisiones).
- $i_j$ es la posición de la palabra actual en la secuencia original y $i_{j -1}$ es la posición de la palabra anterior en el skip-gram.
- $\sum_{j=2}^{n}(i_j -i_{j -1} -1) $ es la suma de las palabras omitidas en el skip-grama completo. Si esta suma es menor o igual a $k$, entonces la secuencia es un k-skip-n-grama válido.
- $w_{i_1}w_{i_2}\dots w_{i_n}$  es una secuencia de palabras seleccionadas para formar un skip-grama.

A continuación, te muestro cómo podríamos calcular skip-gramas en Python usando la definición proporcionada:


In [ ]:
def generate_skip_grams(sequence, n, k):
    skip_grams = []
    for i in range(len(sequence) - n + 1):
        # Esta es la ventana inicial para el n-gram sin skips
        base_gram = sequence[i:i+n]
        skip_grams.append(base_gram)
        for skip in range(1, k+1):
            # Intenta generar skip-grams para esta base añadiendo palabras omitidas
            for j in range(i+n, min(len(sequence), i+n+skip)):
                for l in range(n-1):
                    skip_gram = base_gram[:l+1] + [sequence[j]] + base_gram[l+1:]
                    if skip_gram not in skip_grams:
                        skip_grams.append(skip_gram)
    return skip_grams

# Ejemplo de uso
sequence = ["El", "gato", "come", "pescado"]
n = 2  # Queremos bigramas
k = 1  # Permitimos un skip
skip_grams = generate_skip_grams(sequence, n, k)
for sg in skip_grams:
    print(sg)


Este código generará todos los posibles skip-gramas de acuerdo con los parámetros `n` y `k` dados. En este caso, el parámetro `n` está configurado en 2 para bigramas y `k` está configurado en 1, lo que significa que se permitirá la omisión de una palabra entre las dos palabras del bigrama.

Vamos a extender la función `generate_skip_grams` para manejar trigramas con dos skips:

In [ ]:
def generate_skip_grams(sequence, n, k):
    skip_grams = []
    
    def recurse_gram(base, start, depth):
        if depth == n:
            skip_grams.append(base)
            return
        for i in range(start, min(len(sequence), start + k + 1)):
            recurse_gram(base + [sequence[i]], i + 1, depth + 1)
    
    for i in range(len(sequence)):
        recurse_gram([sequence[i]], i + 1, 1)
    
    return skip_grams

# Ejemplo de uso
sequence = ["El", "gato", "come", "pescado"]
n = 3  # Queremos trigramas
k = 2  # Permitimos dos skips
skip_grams = generate_skip_grams(sequence, n, k)
for sg in skip_grams:
    print(" ".join(sg))


Este código imprimirá todos los posibles 2-skip-trigrams basados en la secuencia dada. La función `recurse_gram` se llama recursivamente para construir los skip-gramas.

### **Ejercicios**

#### **Ejercicio 1: Análisis de frecuencia de bigramas**
Dado un corpus de texto más grande (por ejemplo, un artículo, un capítulo de un libro o una colección de tuits), realiza lo siguiente:

1. Limpia el texto eliminando signos de puntuación y convirtiendo todo a minúsculas.
2. Divide el texto en palabras y calcula la frecuencia de todos los bigramas.
3. Encuentra los 10 bigramas más frecuentes en el texto.
4. Calcula la probabilidad condicional de la segunda palabra de cada bigrama dado la primera palabra.


In [ ]:
## Tu respuesta

#### **Ejercicio 2: Suavizado de Laplace**
Implementa una función de suavizado de Laplace que funcione para cualquier n-grama:

1. Define una función para contar n-gramas de cualquier longitud en un corpus dado.
2. Implementa el suavizado de Laplace en una función que calcule la probabilidad de un n-grama dado.
3. Prueba tu función con un conjunto de datos de ejemplo y varios valores de $n$ y $\beta$.


In [ ]:
## Tu respuesta

#### **Ejercicio 3: Generación de texto con modelos Markovianos**

Construye un modelo generativo de texto simple:

1. Utiliza un conjunto de datos de texto para entrenar un modelo de bigramas con suavizado de Laplace.
2. Escribe una función que genere texto de manera aleatoria, seleccionando la siguiente palabra en función de las probabilidades del modelo.
3. Genera un párrafo de texto y evalúa su coherencia.


In [ ]:
## Tu respuesta

#### **Ejercicio 4: Implementación de backoff**
Amplía el modelo de bigramas para implementar una estrategia de backoff:

1. Implementa una función que calcule la probabilidad de un bigrama utilizando backoff al unigrama cuando no hay suficiente información.
2. Asegúrate de que el suavizado de Laplace esté incluido en tu cálculo.
3. Prueba tu modelo con secuencias de palabras que no estén en tu corpus y compara los resultados con y sin backoff.


In [ ]:
## Tu respuesta

#### **Ejercicio 5: Modelos Skip-gram**
Practica con modelos skip-gram:

1. Escribe una función que genere todos los posibles k-skip-n-gramas para un valor dado de $k$ y $n$.
2. Calcula las frecuencias de estos k-skip-n-gramas en un corpus de texto.
3. Analiza cómo la elección de $k$ afecta la cantidad y la variedad de skip-gramas generados.

In [ ]:
## Tu respuesta